In [2]:
import os
import pandas as pd
import requests as req
import json
import numpy as np
import re
from unidecode import unidecode

MAX_NAME_SIZE = 50

In [3]:
DATA_URL = 'https://api.got.show/api/characters/'

data = req.get(DATA_URL).content

In [4]:
df = pd.read_json(data, orient='records')
len(df)

2028

In [5]:
df = df[['name', 'dateOfBirth', 'dateOfDeath']]

In [6]:
df_only_born = df.dropna(subset=['dateOfBirth'])
len(df_only_born)

478

In [7]:
def build_dict(names):
    out_dict = set()
    
    for name in names:
        for letter in name:
            out_dict.add(letter)
    
    return sorted(list(out_dict))

def preprocess_name(name):
    out_name = unidecode(name.lower())
    out_name = re.sub(r'\W', '', out_name)
    
    if len(out_name) > MAX_NAME_SIZE:
        out_name = out_name[:MAX_NAME_SIZE]
    
    return out_name

def preprocess_age(birth, death):
    if np.isnan(death):
        return -1
    else:
        return int(death - birth)

def names_to_letters_list(names):
    letters_list = []
    
    for name in names:
        letters_list.append(list(name))
    
    return letters_list

In [8]:
df_names = df_only_born['name'].apply(preprocess_name)

In [9]:
LETTERS_DICT = build_dict(list(df_names))

In [10]:
list_ages = []

for birth, death in zip(list(df_only_born['dateOfBirth']), list(df_only_born['dateOfDeath'])):
    list_ages.append(preprocess_age(birth, death))

In [11]:
def onehot_encoder(name):
    name_encoded = np.zeros((MAX_NAME_SIZE, len(LETTERS_DICT)), dtype='int')
    for index, character in enumerate(list(name)):
        char_index = LETTERS_DICT.index(character)
        name_encoded[index, char_index] = 1
    
    return name_encoded

In [12]:
onehot_names = []

for index, name in enumerate(list(df_names)):
    onehot_names.append(onehot_encoder(name))

onehot_names = np.array(onehot_names)
onehot_names.shape

(478, 50, 26)

In [70]:
def encode_vect(name):
    name_encoded = np.zeros((MAX_NAME_SIZE), dtype='int')
    for index, character in enumerate(list(name)):
        char_index = LETTERS_DICT.index(character)
        name_encoded[index] = char_index
    
    return name_encoded

In [71]:
encoded_names = []

for index, name in enumerate(list(df_names)):
    encoded_names.append(encode_vect(name))

encoded_names = np.array(encoded_names)
encoded_names.shape

(478, 50)

In [13]:
from sklearn.preprocessing import MinMaxScaler
mm_scaler = MinMaxScaler()
scaler = mm_scaler.fit([[age] for age in list_ages])

In [28]:
ages_scaled = scaler.transform([[age] for age in list_ages])
ages_scaled = np.array(ages_scaled)
ages_scaled.shape

(478, 1)

# Machine Learning

In [107]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Flatten
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

model = Sequential()
model.add(Embedding(len(LETTERS_DICT), 30, input_length=MAX_NAME_SIZE))
model.add(Flatten())
model.add(Dense(units=20, activation='sigmoid'))
model.add(Dense(units=10, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 50, 30)            780       
_________________________________________________________________
flatten_16 (Flatten)         (None, 1500)              0         
_________________________________________________________________
dense_38 (Dense)             (None, 20)                30020     
_________________________________________________________________
dense_39 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 11        
Total params: 31,021
Trainable params: 31,021
Non-trainable params: 0
_________________________________________________________________


In [110]:
model.compile(optimizer="adam", loss='binary_crossentropy',  metrics=['accuracy'])

history = model.fit(x=encoded_names, y=ages_scaled, epochs=100, verbose=1, shuffle=True)

#score = model.evaluate(x_test, y_test_oh, verbose=0)

#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

Epoch 1/100
478/478 [==============================] - 0s 745us/step - loss: 0.2942 - acc: 0.6569
Epoch 2/100
478/478 [==============================] - 0s 88us/step - loss: 0.2913 - acc: 0.6569
Epoch 3/100
478/478 [==============================] - 0s 89us/step - loss: 0.2896 - acc: 0.6569
Epoch 4/100
478/478 [==============================] - 0s 88us/step - loss: 0.2884 - acc: 0.6569
Epoch 5/100
478/478 [==============================] - 0s 85us/step - loss: 0.2873 - acc: 0.6569
Epoch 6/100
478/478 [==============================] - 0s 78us/step - loss: 0.2860 - acc: 0.6569
Epoch 7/100
478/478 [==============================] - 0s 84us/step - loss: 0.2854 - acc: 0.6569
Epoch 8/100
478/478 [==============================] - 0s 85us/step - loss: 0.2842 - acc: 0.6569
Epoch 9/100
478/478 [==============================] - 0s 88us/step - loss: 0.2826 - acc: 0.6569
Epoch 10/100
478/478 [==============================] - 0s 91us/step - loss: 0.2814 - acc: 0.6569
Epoch 11/100
478/478 [======

478/478 [==============================] - 0s 98us/step - loss: 0.2349 - acc: 0.6590
Epoch 85/100
478/478 [==============================] - 0s 111us/step - loss: 0.2355 - acc: 0.6590
Epoch 86/100
478/478 [==============================] - 0s 87us/step - loss: 0.2344 - acc: 0.6590
Epoch 87/100
478/478 [==============================] - 0s 79us/step - loss: 0.2341 - acc: 0.6590
Epoch 88/100
478/478 [==============================] - 0s 80us/step - loss: 0.2341 - acc: 0.6590
Epoch 89/100
478/478 [==============================] - 0s 85us/step - loss: 0.2335 - acc: 0.6590
Epoch 90/100
478/478 [==============================] - 0s 84us/step - loss: 0.2331 - acc: 0.6590
Epoch 91/100
478/478 [==============================] - 0s 80us/step - loss: 0.2332 - acc: 0.6590
Epoch 92/100
478/478 [==============================] - 0s 85us/step - loss: 0.2324 - acc: 0.6590
Epoch 93/100
478/478 [==============================] - 0s 98us/step - loss: 0.2320 - acc: 0.6590
Epoch 94/100
478/478 [==========

In [131]:
my_name = encode_vect(preprocess_name('Vinícius'))
my_name

array([21,  8, 13,  8,  2,  8, 20, 18,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [132]:
res = model.predict(np.array([my_name]))
print(res[0][0])
scaler.inverse_transform(res[0][0])

0.0540818


array([[ 4.57042074]], dtype=float32)